<h2> Importação de bibliotecas </h2>

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

<h2> Configuração do webdriver e definição de variáveis </h2>

In [ ]:
service = Service()
options = webdriver.ChromeOptions()
options.add_argument("--window-size=1920,1200")
driver = webdriver.Chrome(service=service, options=options)

url = 'https://www.trabalhabrasil.com.br/'
driver.get(url)

busca_filtro = "Desenvolvedor Java"

palavras_chave = [
    "Backend",
    "Java",
    "6"
    "8",
    "11",
    "17",
    "JSP",
    "JSF",
    "Angular",
    "Rest",
    "Soap",
    "JDBC",
    "JPA",
    "Spring",
    "Boot",
    "Security",
    "Hibernate",
    "Sql",
    "Oracle",
    "MySql"
    "Git",
    "Scrum",
    "Frontend",
    "JavaScript",
    "HTML",
    "CSS",
    "Jasper",
    "Monolito",
    "Webservices",
    "ágeis"
]

num_palavras_chave_necessarias = 3
num_vagas_percorridas = 15

wait = WebDriverWait(driver, 10)


<h2> Funções auxiliares </h2>

In [ ]:
# Salva as urls das vagas compativeis em um arquivo txt
def salvar_urls_encontradas(urls_encontradas):
    try:
        with open("urls_vagas_encontradas.txt", "a") as arquivo:
            for url in urls_encontradas:
                arquivo.write(url + "\n")
    except Exception as e:
        print(f"Erro ao salvar as URLs encontradas: {e}")

# Retorna a decricao da vaga
def descricao_vaga(wait):
    elementos_box = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "box")))
    ultimo_elemento_box = elementos_box[-1]
    
    textos_p = []
    tags_p = ultimo_elemento_box.find_elements(By.TAG_NAME, "p")
    for tag_p in tags_p:
        textos_p.append(tag_p.text)
    return textos_p

# Retorna a url atual
def url_atual(driver):
    url_atual = driver.current_url
    return url_atual

# Retorna True caso a descricao da vaga possua compatibilidade com as palavras-chave
def vaga_compativel(wait, palavras_chave):
    try:
        textos_p = descricao_vaga(wait)
        palavras_chave = [palavra.lower() for palavra in palavras_chave]
        palavras_texto = ' '.join(textos_p).split()
        contagem_encontrada = sum(palavra_chave.lower() in palavra_texto.lower() for palavra_texto in palavras_texto for palavra_chave in palavras_chave)
        if contagem_encontrada >= num_palavras_chave_necessarias:
            return True
        else:
            return False
    except Exception as e:
        print(f"Erro ao verificar palavras-chave na descrição da vaga: {e}")
        return None

# Avança para a proxima vaga
def proxima_vaga(wait, driver):
    botao_next_job = wait.until(EC.element_to_be_clickable((By.ID, "nextJobView")))
    driver.execute_script("arguments[0].click();", botao_next_job)

# Altera o drive para o frame da propaganda, fecha a propagandan e retorna para o frame original
def fechar_propaganda():
    try:
        iframe_locator = (By.ID, "google_ads_iframe_/66517371/pg_interstitial_trabalhabrasil.com.br_0")
        iframe = wait.until(EC.presence_of_element_located(iframe_locator))

        if(iframe):
            print("Frame da propaganda encontrado")
            driver.switch_to.frame(iframe)
            print("Frame trocado")
            driver.find_element(By.ID, "dismiss-button").click()
            print("Propaganda fechada")
    except:
        pass
    finally:
        driver.switch_to.default_content()
        print("Frame original setado")

# Realiza a filtragem da busca de vagas
def filtra_busca(busca_filtro, wait, driver):
    input_filtro = wait.until(EC.presence_of_element_located((By.ID, "txtRoleQuery")))
    input_filtro.send_keys(busca_filtro)

    botao_filtro = driver.find_element(By.ID, "btnBuscarVagas")
    botao_filtro.click()

# Seta a opção para apenas homeOffice
def filtra_homeoffice(wait):
    botao_home_office = wait.until(EC.presence_of_element_located((By.ID, "btnBuscarHome")))
    botao_home_office.click()

# Percorre as vagas na quantidade setada em 'num_vagas_percorridas' e salva num arquivo txt
def percorre_vagas(wait, driver, palavras_chave, num_vagas_percorridas):
    cards_job = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "jobCard")))[0]
    cards_job.click()
    
    urls_encontradas = []

    for i in range(num_vagas_percorridas):
        try:
            compativel = vaga_compativel(wait, palavras_chave)
            if compativel:
                url = url_atual(driver)
                urls_encontradas.append(url)
                print("url salva: " + url)
            
            proxima_vaga(wait, driver)
        except:
            fechar_propaganda()

    salvar_urls_encontradas(urls_encontradas)

<h2> Execução do webscraping </h2>

In [ ]:
filtra_busca(busca_filtro, wait, driver)

filtra_homeoffice(wait)

percorre_vagas(wait, driver, palavras_chave, num_vagas_percorridas)
